In [6]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests


In [ ]:
conf = SparkConf()
conf.setMaster('local[2]')
conf.setAppName("TwitterStreamApp1")
sc = SparkContext(conf=conf)
sc.setLogLevel("Error")
ssc = StreamingContext(sc, 1)
ssc.checkpoint("checkpoint_TwitterApp")
dataStream = ssc.socketTextStream("localhost",9009)

def sum_tags_count(new_values,total_sum):
    return sum(new_values)+(total_sum or 0)
def get_sql_context_instance(spark_context):
    if('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']
def process_rdd(time,rdd):
    print("-----------%s-----------"%str(time))
    try:
        #Get spark sql singleton context from the current context
        sql_context=get_sql_context_instance(rdd.context)
        #converttheRDDtoRowRDD
        row_rdd=rdd.map(lambda w:Row(hashtag=w[0],hashtag_count=w[1]))
        #create a DF from the Row RDD
        hashtags_df = sql_context.createDataFrame(row_rdd)
        #Register the dataframe as table
        hashtags_df.registerTempTable("hashtags")
        #get the top10 hashtags from the table using SQL and print them
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags where hashtag in \
                                            ('#movie', '#Trump','#game', '#girl', '#art', '#dog', '#Dota'\
                                             '#cs', '#music', '#class')")
        hashtag_counts_df.show()
    except:
        e=sys.exc_info()[0]
        print(e)
    #finally:
        #hashtag_counts_df.show()

In [ ]:
words = dataStream.flatMap(lambda line: line.split(" "))
hashtags = words.filter(lambda w: '#' in w).map(lambda x:(x, 1))
tags_totals = hashtags.updateStateByKey(sum_tags_count)
tags_totals.foreachRDD(process_rdd)
ssc.start()
ssc.awaitTermination()